In [60]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from openai.types.chat.completion_create_params import WebSearchOptions
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from openai import OpenAI
from typing import Dict
from IPython.display import display, Markdown
from bs4 import BeautifulSoup
from firecrawl import Firecrawl

import asyncio
import os
import requests
import json
import os

from prompts.research_prompts import (
    COMPANY_STRATEGY_SYSTEM_PROMPT, 
    ROLE_SUCCESS_SYSTEM_PROMPT, 
    TEAM_CULTURE_SYSTEM_PROMPT, 
    DOMAIN_KNOWLEDGE_SYSTEM_PROMPT, 
    company_strategy_query, 
    role_success_query, 
    team_culture_query, 
    domain_knowledge_query,
)
from prompts.job_parsing_prompts import JOB_LISTING_RESEARCH_PROMPT_V1
from prompts.distillation_prompts import DISTILLATION_SYSTEM_PROMPT_V1

In [61]:
load_dotenv(override=True)
openai = OpenAI()

## Step 1 - Fetch Job Details (Firecrawl)

In [62]:
JOB_LISTING_URL = "https://www.indeed.com/viewjob?jk=8a3614a059560345"
# JOB_LISTING_URL = "https://edwarddonner.com"

In [63]:
firecrawl = Firecrawl(api_key=os.getenv("FIRECRAWL_API_KEY"))

In [64]:
# FETCHED VERSION
# scraped_job_listing = firecrawl.scrape(JOB_LISTING_URL, formats=["markdown"]).markdown
# with open("./saved_texts/scraped_job.md", "w", encoding="utf-8") as f:
#     f.write(scraped_job_listing.markdown)

# CACHED VERSION
with open("./saved_texts/scraped_job.md", "r", encoding="utf-8") as f:
    scraped_job_listing = f.read()

scraped_job_listing

'WhatWhereFind Jobs\n\nThis job has expired on Indeed\n\nReasons could include: the employer is not accepting applications, is not actively hiring, or is reviewing applications\n\n# Account Manager\n\n[Coca-Cola Northeast](https://www.indeed.com/cmp/Coca--cola-Beverages-Northeast?campaignid=mobvjcmp&from=mobviewjob&tk=1j982d9ilgmu4802&fromjk=8a3614a059560345)\n\n3.33.3 out of 5 stars\n\nWestborough, MA 01581\n\n$70,000 a year\n\n[**Coca-Cola Northeast** \\\\\n\\\\\n228 reviews\\\\\nRead what people are saying about working here.](https://www.indeed.com/cmp/Coca--cola-Beverages-Northeast/reviews?campaignid=mobvjcmp&cmpratingc=mobviewjob&from=mobviewjob&tk=1j982d9ilgmu4802&fromjk=8a3614a059560345&jt=Account+Manager)\n\n## Benefits  Pulled from the full job description\n\n- Referral program\n- Tuition reimbursement\n- Health insurance\n- 401(k) matching\n- Paid time off\n- Employee discount\n- Life insurance\n\n&nbsp;\n\n## Full job description\n\n- Compensation: $70,000\n- Schedule: Mond

In [65]:
JOB_LISTING_RESEARCH_PROMPT_V1

'ROLE: You are a senior research-analyst LLM that extracts relevant information from a job listing website and organizes it into a JSON breakdown of key attributes.\n\n# INPUT: \nYou will receive the scraped contents of a job listing website in markdown format.\n\n# TASK\nParse through the scraped contents to extract ALL relevant attributes as they appear on the website.\n\n# GLOBAL RULES\n- Never fabricate. If a field cannot be found, use "Unknown".\n- NEVER summarize or shorten. Use the exact text from the website.\n- ALWAYS use bullet points for "expectations_and_responsibilities" and "requirements".\n\n# JOB ATTRIBUTES\n- job_title\n  - Verbatim job title from website.\n- job_location\n  - City, state, and zip code (if available). May be "remote".\n- job_description\n  - Verbatim description from website.\n- work_schedule \n  - Days and hours you are expected to work.\n- company_name\n  - Verbatim company name from website.\n- expectations_and_responsibilities\n  - BULLETED list an

In [66]:
class JobListingResearchResponse(BaseModel):
    job_title: str
    job_location: str
    job_description: str
    work_schedule: str
    company_name: str
    expectations_and_responsibilities: str
    requirements: str

def extract_job_listing_attributes(scraped_job_contents: str) -> JobListingResearchResponse:
    response = openai.beta.chat.completions.parse(
        model="gpt-4.1-nano", 
        messages=[
            {"role": "system", "content": JOB_LISTING_RESEARCH_PROMPT_V1},
            {"role": "user", "content": f"Scraped website contents: \n\n{scraped_job_contents}"}
        ],
        temperature=0.3,
        top_p=0.9,
        response_format=JobListingResearchResponse,
    )

    # Parse the JSON response content into a JobListingResearchResponse instance
    content = response.choices[0].message.content
    response_data = json.loads(content)
    return JobListingResearchResponse(**response_data)

In [67]:
job_listing_research_response = extract_job_listing_attributes(scraped_job_listing)
job_listing_research_response

KeyboardInterrupt: 

In [ ]:
# print(json.dumps(job_listing_research_response.model_dump(), indent=4, sort_keys=True))
print(job_listing_research_response.expectations_and_responsibilities)
print("====")
print(job_listing_research_response.requirements)

- Educate and excite customers about our leading portfolio of beverages
- Analyze customer needs and business trends to help grow both their profitability and our own business
- Ensure brands and packages sold align with an account’s consumer demographics
- Sell incremental displays, equipment placements and promotional programs
- Maintain appropriate inventory levels, company assets, promotional ad signage (point of sale) and displays
- Ensure each account meets defined merchandising standards, determine stores’ product needs and transmit appropriate replenishment orders
- Communicate and collaborate account activities to appropriate business partners
- Utilize sales force automation tools
- Work with customers to promptly address and resolve any concerns or discrepancies pertaining to their orders
====
- High school diploma, GED or equivalent required
- Bachelor’s degree preferred
- 1+ years of relevant sales experience preferred
- Must have the ability to regularly lift, push and pu

## Step 2 - Deep Research

In [ ]:
company_strategy_agent = Agent(
    name="Company strategy agent",
    instructions=COMPANY_STRATEGY_SYSTEM_PROMPT,
    tools=[WebSearchTool(search_context_size="medium")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

role_success_agent = Agent(
    name="Role success agent",
    instructions=ROLE_SUCCESS_SYSTEM_PROMPT,
    tools=[WebSearchTool(search_context_size="medium")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

team_culture_agent = Agent(
    name="Team culture agent",
    instructions=TEAM_CULTURE_SYSTEM_PROMPT,
    tools=[WebSearchTool(search_context_size="medium")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

domain_knowledge_agent = Agent(
    name="Domain knowledge agent",
    instructions=DOMAIN_KNOWLEDGE_SYSTEM_PROMPT,
    tools=[WebSearchTool(search_context_size="medium")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [ ]:
async def perform_research_simultaneously(job_listing_research_response: JobListingResearchResponse):
    company_name = job_listing_research_response.company_name
    job_title = job_listing_research_response.job_title

    research_tasks = [
        Runner.run(company_strategy_agent, company_strategy_query(company_name)),
        Runner.run(role_success_agent, role_success_query(company_name, job_title)),
        Runner.run(team_culture_agent, team_culture_query(company_name, job_title)),
        Runner.run(domain_knowledge_agent, domain_knowledge_query(company_name, job_title)),
    ]

    results = await asyncio.gather(*research_tasks, return_exceptions=False)

    return results

In [ ]:
trace_id = gen_trace_id()
with trace(f"Deep Job Research: {trace_id}"):
    deep_research_results = await perform_research_simultaneously(job_listing_research_response)
    
deep_research_results

[RunResult(input='site:Coca-Cola Northeast.com OR ("Coca-Cola Northeast" profile OR overview OR mission OR leadership OR strategy OR values OR "business model" OR "products and services" OR "competitors" OR "market position") latest news OR press release OR annual report OR funding OR growth OR acquisitions', new_items=[ToolCallItem(agent=Agent(name='Company strategy agent', handoff_description=None, tools=[WebSearchTool(user_location=None, filters=None, search_context_size='medium')], mcp_servers=[], mcp_config={}, instructions='ROLE: You are a senior research-analyst LLM that fetches web results from a given query and produces a single, rigorously structured Markdown report on a company in order to prepare for an upcoming job interview. Be precise, source-driven, and concise.\n\n# INPUT (JSON): \n{\n "job_title": <tile_of_role_interviewing_for>,\n "job_location": <city_state_zip_code_or_remote>,\n "company": <company_that_is_hiring>,\n "web_search_query": <web_search_query_to_run>,\n

In [ ]:
# for result in deep_research_results:
#     print(result.final_output)
#     print("====")
display(Markdown(deep_research_results[3].final_output))


# Function-Specific and Domain Knowledge

## Functional Overview

- **Department or Function**: Sales and Marketing
- **Strategic Purpose**: To drive revenue growth by developing and maintaining strong relationships with key customers, ensuring product availability, and executing effective marketing strategies.
- **Key Workstreams or Initiatives**:
  - **Key Account Management**: Collaborating with major customers to enhance profit margins through increased volume and revenue, while reducing distribution costs.
  - **Revenue Growth Management (RGM)**: Implementing strategies across various markets to focus on value over volume, including innovation and premium offerings.
  - **Digital Transformation**: Expanding digital marketing efforts and e-commerce capabilities to engage consumers across both physical and digital touchpoints.

## Tools, Frameworks, and Practices

- **Common Frameworks, Systems, or Methodologies Used**:
  - **Collaborative Key Account Management**: Building long-term relationships with major customers to improve profit margins.
  - **Revenue Growth Management (RGM)**: Focusing on value over volume to drive revenue growth.
  - **Digital Marketing Strategies**: Utilizing voice sampling campaigns, chatbot technology, and digitally enabled packaging.
- **Domain-Specific Tools / Tech Stack**:
  - **Enterprise Data Warehouse (EDW)**: Centralizing data to provide a single view of multichannel retail data.
  - **Big Data Analytics**: Analyzing large volumes of data to make informed decisions about products, sales revenue, and customer experiences.
- **Internal Standards or Best Practices**:
  - **On-Time In-Full (OTIF) Delivery**: Ensuring orders are delivered on time and in full, with a target of over 90% OTIF.
  - **Collaborative Planning, Forecasting, and Replenishment (CPFR)**: Working closely with customers to improve forecasting and inventory management.
- **Why this matters for interviews**: Understanding these frameworks and tools is crucial for demonstrating how you can contribute to Coca-Cola's strategic initiatives and operational excellence.

## Industry / Domain Concepts

- **Common Terminology or Abbreviations**:
  - **RGM**: Revenue Growth Management
  - **OTIF**: On-Time In-Full
  - **CPFR**: Collaborative Planning, Forecasting, and Replenishment
- **Foundational Concepts or Models**:
  - **Collaborative Key Account Management**: A strategy focused on building strong, long-term relationships with major customers to drive mutual growth.
  - **Revenue Growth Management (RGM)**: A framework that emphasizes value over volume to achieve sustainable revenue growth.
- **Current Innovations or Trends in the Domain**:
  - **Digital Transformation**: Leveraging digital tools and platforms to enhance customer engagement and streamline operations.
  - **Data Analytics**: Utilizing big data to inform decision-making and improve business performance.
- **Why this matters for interviews**: Familiarity with these concepts will enable you to align your experience with Coca-Cola's strategic direction and demonstrate your ability to contribute effectively.

## Challenges and Opportunities

- **Known Industry or Role-Specific Pain Points**:
  - **Obesity Concerns**: Addressing public health concerns related to obesity by offering healthier beverage options.
  - **Intense Competition**: Navigating a highly competitive beverage market with numerous global and local players.
- **Areas Where Innovation or Efficiency Is Needed**:
  - **Supply Chain Optimization**: Enhancing supply chain efficiency to reduce costs and improve service levels.
  - **Digital Engagement**: Expanding digital marketing and e-commerce capabilities to meet evolving consumer preferences.
- **High-Impact Opportunities**:
  - **Health-Conscious Products**: Developing and promoting beverages that cater to health-conscious consumers.
  - **E-Commerce Growth**: Capitalizing on the growth of online sales channels to reach a broader audience.
- **Why this matters for interviews**: Understanding these challenges and opportunities will help you articulate how your skills and experiences can address Coca-Cola's current needs and future goals. 

In [ ]:
with open("./saved_texts/deep_research_results.md", "w") as f:
    f.write("\n\n".join(result.final_output for result in deep_research_results))

AttributeError: 'str' object has no attribute 'final_output'

In [ ]:
with open("./saved_texts/deep_research_results.md", "r") as f:
    deep_research_results = f.read()

deep_research_results

'# Company Context & Strategy\n\n## Identity & Overview\n\n- **Company Name**: Coca-Cola Consolidated, Inc.\n- **Mission / Vision Statements**: While specific mission and vision statements are not publicly disclosed, Coca-Cola Consolidated focuses on delivering high-quality beverages and maintaining strong customer relationships. ([kabuseek.s3.ap-northeast-1.amazonaws.com](https://kabuseek.s3.ap-northeast-1.amazonaws.com/filing/DPS/2012-02-22/dps-20111231x10k.htm?utm_source=openai))\n- **Growth Stage / Size / Funding Snapshot**: As of the fiscal year ending December 31, 2023, Coca-Cola Consolidated reported revenues of $6.6539 billion, reflecting a 7.3% increase from the previous year. ([coursehero.com](https://www.coursehero.com/tutors-problems/Business-Other/63121183-Executive-Summary-Coca-Cola-Consolidated-Inc-CCCI-is-a/?utm_source=openai))\n- **As-of Date**: November 4, 2025\n\n## Strategy & Positioning\n\n- **Core Offerings**:\n  - **Carbonated Soft Drinks (CSDs)**: Including flag

## Step 3 - Distill Research into Interview Guide

In [ ]:
with open("./saved_texts/interview_questions.md", "r") as f:
    interview_questions = f.read()

interview_questions

"Job-specific\n- What: Questions that test the exact skills, tools, and domain knowledge used in the job.\n- Why companies ask: To verify you can do the day-to-day work with minimal ramp-up.\n- How to prepare: Study the job description; review core concepts, systems, and tools; practice realistic tasks (e.g., coding, mock tickets, sample deliverables); bring concise, concrete examples from your past work.\n- Examples:\n  - What are the advantages and disadvantages of the accounting software you used in your last job?\n    - Assesses familiarity with industry tools and ability to evaluate technology trade-offs; common in accounting, finance, or bookkeeping roles.\n  - Are you comfortable using a telephone with several lines and handling high call volume?\n    - Tests multitasking and communication under pressure; typical for receptionists, administrative assistants, and call center representatives.\n  - What do you like and dislike about working in advertising?\n    - Explores motivatio

In [ ]:
interview_guide_input = "\n".join([
    f"job_title: {job_listing_research_response.job_title}",
    f"job_description: {job_listing_research_response.job_description}",
    f"company_name: {job_listing_research_response.company_name}",
    f"\ndeep_research_results: \n{deep_research_results}",
    f"\ninterview_questions: \n{interview_questions}",
])

print(interview_guide_input)

job_title: Account Manager
job_description: The Account Manager serves as the primary contact between Coke Northeast management and store owners within an assigned territory. In this role, the Account Manager will thrive in a dynamic work environment where collaboration with distributors, retailers, and internal teams is essential for developing and implementing effective sales strategies, fostering strong relationships, and driving growth in a competitive market.
company_name: Coca-Cola Northeast

deep_research_results: 
# Company Context & Strategy

## Identity & Overview

- **Company Name**: Coca-Cola Consolidated, Inc.
- **Mission / Vision Statements**: While specific mission and vision statements are not publicly disclosed, Coca-Cola Consolidated focuses on delivering high-quality beverages and maintaining strong customer relationships. ([kabuseek.s3.ap-northeast-1.amazonaws.com](https://kabuseek.s3.ap-northeast-1.amazonaws.com/filing/DPS/2012-02-22/dps-20111231x10k.htm?utm_source

In [ ]:
def create_interview_guide(job_listing_research_response: JobListingResearchResponse, deep_research_results: str, interview_questions: str):
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": DISTILLATION_SYSTEM_PROMPT_V1},
            {"role": "user", "content": interview_guide_input},
        ],
        temperature=0.6,
    )

    return response.choices[0].message.content

In [ ]:
interview_guide = create_interview_guide(job_listing_research_response, deep_research_results, interview_questions)

In [ ]:
display(Markdown(interview_guide))

### 1) Role Snapshot

The Account Manager at Coca-Cola Northeast is responsible for driving sales growth and maintaining robust relationships with key accounts in a designated territory. Success is defined by the effective execution of sales plans, relationship management, and achieving primary KPIs such as sales volume, revenue growth, and customer satisfaction.

### 2) Strategic Context — High-Yield Facts

* Coca-Cola Northeast focuses on enhancing brand presence and expanding product offerings to meet consumer preferences.
* The company reported $6.6539 billion in revenue, a 7.3% increase, reflecting robust market performance.
* Key competitors include PepsiCo and Dr Pepper Snapple Group, indicating a highly competitive landscape.
* The shift towards healthier beverage options and sustainability is impacting product offerings and marketing strategies.
* The role requires collaboration with internal teams (Sales, Marketing, Operations) and external partners (retailers, distributors).
* Key tools include CRM software (e.g., Salesforce) and sales analytics platforms to manage account performance.
* The market focus is primarily in the United States, particularly in the Northeast region.
* Success metrics include market share expansion and the effectiveness of promotional activities.

### 3) High-Impact Topics to Probe

* **Sales Strategy:** *Understanding effective sales plans is critical for driving revenue growth.*
* **Relationship Management:** *Strong relationships with key accounts lead to customer retention and satisfaction.*
* **Market Trends:** *Awareness of industry trends helps adapt strategies to meet consumer demands.*
* **Collaboration:** *Effective teamwork is essential for aligning sales efforts with marketing and operations.*
* **Analytical Skills:** *Ability to assess sales data directly impacts decision-making and strategy optimization.*

### 4) Tailored Question Bank

* **Job-Specific (3):**
  - How do you develop and execute annual sales plans for key accounts?
  - What tools do you use for volume forecasting and promotional planning?
  - Can you explain your approach to optimizing trade marketing investments?

* **Behavioral (3):**
  - Describe a time when you successfully turned around a challenging client relationship.
  - What is the most significant sales target you achieved, and how did you reach it?
  - Can you share an experience where teamwork was crucial to achieving a sales goal?

* **Situational (3):**
  - If a key account is unhappy with a recent promotion, how would you address their concerns?
  - How would you prioritize competing demands from multiple key accounts?
  - What steps would you take if you identified a significant drop in sales for one of your accounts?

* **Culture/Values (1):**
  - What attracts you to Coca-Cola Northeast, and how do you align with its mission?

### 5) Signals of Strong Answers

* **Sales Strategy — Signals:**
  - Clear articulation of specific sales strategies and their outcomes.
  - Evidence of adaptability to changing market conditions.
  - Use of metrics to assess and refine sales plans.

* **Relationship Management — Signals:**
  - Examples of long-term relationships built with key accounts.
  - Strategies for maintaining communication and trust.
  - Demonstrated ability to resolve conflicts and enhance partnerships.

* **Market Trends — Signals:**
  - Knowledge of current beverage industry trends and consumer preferences.
  - Ability to pivot strategies based on market insights.
  - Awareness of competitor actions and their implications.

* **Collaboration — Signals:**
  - Examples of cross-functional projects and their outcomes.
  - Evidence of effective communication skills with internal teams.
  - Ability to influence stakeholders and align goals.

* **Analytical Skills — Signals:**
  - Use of data to support decision-making and strategy development.
  - Examples of how analytics led to improved sales performance.
  - Comfort with relevant tools and methodologies for analysis.

### 6) Follow-Up Patterns (Global)

* **Quantify Impact:** “What metric moved, by how much, and over what period?”
* **Stakeholders & Trade-offs:** “Who owned what? What did you trade off and why?”
* **Data & Tooling:** “What inputs did you need, and how did you get them?”
* **Risk & Mitigation:** “What went wrong or could have? How did you prevent/escalate?”

### 7) Micro-Glossary

* **CRM:** *Customer Relationship Management software used to manage interactions with current and potential customers.*
* **RGM:** *Revenue Growth Management, focusing on value over volume to drive revenue growth.*
* **OTIF:** *On-Time In-Full, a metric for order delivery performance.*
* **CPFR:** *Collaborative Planning, Forecasting, and Replenishment, a strategy for improving forecasting accuracy.*
* **CSDs:** *Carbonated Soft Drinks, a primary product category for Coca-Cola.*

### 8) Red Flags to Watch

* Vague or generic responses lacking specific examples related to sales strategies.
* Inability to articulate how previous experiences align with Coca-Cola's mission and values.
* Lack of familiarity with industry trends and competitive landscape.
* Weakness in demonstrating collaboration or relationship-building skills.
* Failure to quantify past successes or understand key performance metrics.

In [ ]:
with open("./saved_texts/interview_guide.md", "w") as f:
    f.write(interview_guide)

## Step 4 - Conducting the Interview

In [ ]:
# 1 - Create system prompt
# 2 - Create gradio chat interface to interact w/ chat
# 3 - Test results are satisfactory (maybe use Atlas agent mode)

## Step 5 - Evaluating Performance

In [ ]:
# 1 - Brainstorm how to evaluate
# 2 - Create system prompt(s)
# 3 - Try it out a couple times

## Step 6 - Create App

In [ ]:
# 1 - Make sure I can do this all in JS
# 2 - Explore pricing for hosting
# 3 - Make the app


# ?? - Add some sort of guardrail for the user inputs?